In [1]:
import import_ipynb
from datetime import datetime
import copy
import numpy as np
import random
import graph_generator as gg
import networkx as nx
import time
from time import perf_counter

In [4]:
def add_neighbours(tree, visited):
    new_neighbours = []
    for node in visited:
        for edge in tree[node]:
            if edge not in visited:
                #print(node, edge)
                new_neighbours.append((node, edge))
    return new_neighbours

In [5]:
def initialize_solution(tree, k):
    start_node = random.randint(0, len(tree)-1)
    solution = [False for _ in range(len(tree))]
    solution[start_node] = True
    visited = [start_node]
    neighbours = add_neighbours(tree, visited)
    weight = 0
    edges = []

    counter = 1
    while counter < k:
        chosen = random.randint(0, len(neighbours)-1)
        visited.append(neighbours[chosen][1])
        edges.append((neighbours[chosen][0], neighbours[chosen][1]))
        solution[neighbours[chosen][1]] = True
        weight += tree[neighbours[chosen][0]][neighbours[chosen][1]]['weight']

        neighbours = add_neighbours(tree, visited)
        counter += 1

    return solution, edges, weight

In [7]:
def add_neighbours_except_k(tree, nodes_in_tree, k):
    new_neighbours = []
    for node in nodes_in_tree:
        if node != k:
            for neighbour in tree[node]:
                if neighbour not in nodes_in_tree:
                    new_neighbours.append((node, neighbour))
    return new_neighbours

In [8]:
def find_candidates(tree, nodes_in_tree, edges):
    candidates = []
    num_of_connections = dict()
    for node in tree:
        num_of_connections[node] = 0
    
    for edge in edges:
        num_of_connections[edge[0]] += 1
        num_of_connections[edge[1]] += 1

    for connection in num_of_connections:
        if num_of_connections[connection] == 1:
            candidates.append(connection)
        
    
    return candidates

In [10]:
def invert_first_improvement(tree, solution, weight, edges):
    new_solution = copy.deepcopy(solution)
    new_weight = weight
    improved = True
    while improved:
        improved = False
        nodes_in_tree = []
        for i in range(len(new_solution)):
            if new_solution[i]:
                nodes_in_tree.append(i)
        candidates = find_candidates(tree, nodes_in_tree, edges)
        chosen_candidate = random.randint(0, len(candidates)-1) #ovde ponekad vrati da je range (0, 0, 0) i puca
        neighbours = add_neighbours_except_k(tree, nodes_in_tree, candidates[chosen_candidate])

        new_solution[candidates[chosen_candidate]] = False
        new_edges = []
        for edge in edges:
            if edge[1] != candidates[chosen_candidate]:
                new_edges.append(edge)
            else:
                new_weight -= tree[edge[0]][edge[1]]['weight']
        if len(neighbours) == 0:
            break
        chosen_neighbour = random.randint(0, len(neighbours)-1)
        new_weight += tree[neighbours[chosen_neighbour][0]][neighbours[chosen_neighbour][1]]['weight']
        new_solution[neighbours[chosen_neighbour][1]] = True
        new_edges.append((neighbours[chosen_neighbour][0], neighbours[chosen_neighbour][1]))

        if new_weight < weight:
            weight = new_weight
            solution = new_solution
            edges = copy.deepcopy(new_edges)
            improved = True
        else:
            new_weight = weight
            new_solution = solution

    return solution, edges, weight
        

In [12]:
def shaking(tree, k):
    new_solution = initialize_solution(tree, k)
    return new_solution

In [13]:
vns_params = {
    'time_limit': 2,
    'l_min': 1,
    'l_max': 2,
    'move_prob': 0.5,
}

In [14]:
def vns_algorithm(tree, k, vns_params):
    start_time = perf_counter()
    solution, edges, weight = initialize_solution(tree, k)
    while perf_counter() - start_time < vns_params['time_limit']:
        for l in range(vns_params['l_min'], vns_params['l_max']):
            new_solution, new_edges, new_weight = shaking(tree, k) #diverzifikacija
            new_solution, new_edges, new_weight = invert_first_improvement(tree, new_solution, new_weight, new_edges) #intenzifikacija

            if new_weight < weight or (new_weight == weight and random.random() < vns_params['move_prob']):
                solution = new_solution
                weight = new_weight
                edges = copy.deepcopy(new_edges)

    return solution, edges, weight
            

In [16]:
def vns(graph, k, time_limit):
    start = time.time()
    vns_params = {
    'time_limit': time_limit,
    'l_min': 1,
    'l_max': 2,
    'move_prob': 0.5,
    }
    tree = nx.minimum_spanning_tree(graph)
    solution, edges, weight = vns_algorithm(tree, k, vns_params)
    end = time.time()
    print(edges, weight)
    print(end-start)
    
    return edges